# Cuaderno 20: Problema del árbol generador de peso mínimo 
# (Minimum Spanning Tree - MST)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo no dirigido $G=(V,E)$; y,
* un vector $c \in \ZZ^{E}$ de costos asociados a las aristas de $G$.

El *problema del árbol generador de peso mínimo* consiste en encontrar un subgrafo $H$ de $G$ con las siguientes características: 
* $H$ debe ser un *árbol*, es decir, debe ser conexo y no contener ciclos; 
* $H$ debe ser un subgrafo *generador* de $G$, es decir, debe contener todos los nodos de $G$; y,
* $H$ debe tener el menor costo posible, donde el costo de $H$ se define como $c(H) = \sum_{ij \in E(H)} c_{ij}$.

Se puede demostrar que las dos primeras propiedades son equivalentes a requerir que $H$ sea conexo, contenga todos los nodos de $G$ y tenga máximo $n - 1$ aristas, donde $n$ es el número de nodos del grafo. A su vez, la condición de que un subgrafo sea conexo y contenga todos los nodos de $G$ equivale a decir que para todo $W \subset V$, $\emptyset \neq W \neq V$, $H$ contiene al menos una arista del *corte* $\delta(W):= \{ ij \in E \, : \, i \in W, j \not\in W\}$.

Utilizando variables binarias $x_{ij}$ para indicar la selección de aristas en $H$, el problema del árbol generador de peso mínimo puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{ij \in E} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{ij \in E} x_{ij} \leq n - 1, \\
&\sum_{ij \in \delta(W)} x_{ij} \geq 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall ij \in E.
\end{align*}

Notar que para establecer la condición de conexidad se requiere de un número exponencial de restricciones con las relación a la cantidad de nodos del grafo.
Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los datos. Usaremos la función `randint` del módulo `random` para generar valores de costos de los arcos aleatoriamente en el rango {-10,..,10}.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm

# Número de nodos del grafo
n = 10

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Aristas del grafo
E = gp.tuplelist([ (1, 3), (1,  5), (1, 6), (1,  7),
                (2, 3), (2,  8), (2, 9), (2, 10),
                (3, 6), (3,  7), (4, 5), (4,  6),
                (4, 9), (4, 10), (5, 6), (5,  8), 
                (7, 8), (7, 10), (8, 9), (9, 10)])

# Costos de las aristas
c={(i,j) : rm.randint(-10,10) for (i,j) in E}


Representamos esta instancia gráficamente empleando funciones de los módulos `networkx` y `ipycytoscape`:

In [ ]:
import networkx as nx
import ipycytoscape
D = nx.Graph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= str(i)
D.add_edges_from(E)
for i,j in E:
    D.edges[i,j]['etiq'] = str(c[i,j])
    D.edges[i,j]['color'] =  '#9dbaea' #if vx[i,j]<=0.1 else '#ff007f'
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=False)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

Vamos a necesitar construir una restricción de corte para cada subconjunto de nodos $W \subset V$ tal que $\emptyset \neq W \neq V$. Con esta finalidad, definimos una función `powerset` empleando las funciones `chain` y `combinations` del módulo `itertools`.

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# Ejemplo:
for W in powerset([1,2,3]):
    print(W)

Definimos ahora el objeto modelo, las variables y la función objetivo:

In [ ]:
# Crear el objeto modelo
m = gp.Model('mst')

# Crear las variables
x = m.addVars(E, name="x", vtype=GRB.BINARY)

# Crear la función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)


Añadimos la restricción de cardinalidad del conjunto de aristas:

In [ ]:
# Restricción de cardinalidad
m.addConstr(x.sum('*','*')  <= n-1,  "cardinalidad")

Para construir las restricciones de corte, usamos la función `powerset` para iterar sobre todos los subconjuntos de nodos $W$ tales que $\emptyset \neq W \neq V$. Para cada subconjunto, se construye primero el conjunto `Wc` con su 
complemento, es decir, con los nodos de $V$ que no están en $W$. La expresión `x.sum(W, Wc)` construye la suma de las variables asociadas a aristas que tienen el "primer" extremo en $W$ y el otro extremo fuera de $W$. De manera similar, `x.sum(Wc, W)` construye la suma de las variables asociadas a aristas que tienen el "primer" extremo fuera de $W$ y el otro extremo en $W$. La suma de ambas expresiones es igual a la suma de todas las aristas del corte.

In [ ]:
# Restricciones de corte
for W in powerset(V):
    if len(W)!=0 and len(W)!=len(V):
        Wc = [i for i in V if i not in W]
        # Como el grafo es no dirigido, el corte contiene las aristas en sum(W, Wc) y sum(Wc, W)
        m.addConstr(x.sum(W, Wc) + x.sum(Wc, W) >= 1, 
                        "corte[{}]".format(W))


Escribimos el modelo a un archivo de texto:

In [ ]:
# Escribir el modelo a un archivo
m.write('mst.lp')


Finalmente, resolvemos el modelo y mostramos la solución:

In [ ]:
# Calcular la solución óptima
m.optimize()

# Escribir la solución
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nArbol generador de peso mínimo:')
    for i,j in E:
        if vx[i,j] >= 0.9:
            print('{} - {}'.format(i, j))


Dibujamos la solución usando las funciones de los módulos `networkx` y `ipycytoscape`:

In [ ]:
import networkx as nx
import ipycytoscape
D = nx.Graph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= str(i)
D.add_edges_from(E)
for i,j in E:
    D.edges[i,j]['etiq'] = str(c[i,j])
    D.edges[i,j]['color'] =  '#9dbaea' if vx[i,j]<=0.1 else '#ff007f'
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=False)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del árbol generador de peso mínimo (MST)

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
from itertools import chain, combinations
import networkx as nx
import ipycytoscape


def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# Número de nodos del grafo
n = 10

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Aristas del grafo
E = gp.tuplelist([ (1, 3), (1,  5), (1, 6), (1,  7),
                (2, 3), (2,  8), (2, 9), (2, 10),
                (3, 6), (3,  7), (4, 5), (4,  6),
                (4, 9), (4, 10), (5, 6), (5,  8), 
                (7, 8), (7, 10), (8, 9), (9, 10)])

# Costos de las aristas
c={}
for i,j in E:
    c[i,j] = rm.randint(-10,10)

try:
    # Crear el objeto modelo
    m = gp.Model('mst')

    # Crear las variables
    x = m.addVars(E, name="x", vtype=GRB.BINARY)

    # Crear la función objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # Restricción de cardinalidad
    m.addConstr(x.sum('*','*')  <= n-1,  "cardinalidad")

    # Restricciones de corte
    PV = list(powerset(V))
    for W in PV:
        if len(W)!=0 and len(W)!=len(V):
            Wc = [i for i in V if i not in W]
            m.addConstr(x.sum(W, Wc) + x.sum(Wc, W) >= 1, 
                        "corte[{}]".format(W))

    # Escribir el modelo a un archivo
    m.write('mst.lp')

    # Calcular la solución optima
    m.optimize()

    # Escribir la solución
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nArbol generador de peso mínimo:')
        for i,j in E:
            if vx[i,j] > 0:
                print('{} - {}'.format(i, j))
                
    # Graficar la solución
    D = nx.Graph()
    D.add_nodes_from(V)
    for i in V:
        D.nodes[i]['etiq']= str(i)
    D.add_edges_from(E)
    for i,j in E:
        D.edges[i,j]['etiq'] = str(c[i,j])
        D.edges[i,j]['color'] =  '#9dbaea' if vx[i,j]<=0.1 else '#ff007f'
    grafo = ipycytoscape.CytoscapeWidget()
    grafo.graph.add_graph_from_networkx(D, directed=False)
    grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
                
except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')